In [1]:
import os
import pandas as pd
import folium

# 경로 설정
current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
DATA_DIR = os.path.join(project_root, 'data', 'processed')
DATA_DIR2 = os.path.join(project_root, 'data', 'modeling')

# 1. 파일 존재 여부 체크
stations_path = os.path.join(DATA_DIR, 'charging_stations_seoul_gridded.csv')
grids_path = os.path.join(DATA_DIR, 'grid_system_processed.csv')
optimal_path = os.path.join(DATA_DIR2, 'mclp_selected_optimal.csv')

for path in [stations_path, grids_path, optimal_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"파일이 존재하지 않습니다: {path}")

# 2. 데이터 불러오기
charging_stations = pd.read_csv(stations_path)
grids = pd.read_csv(grids_path)
optimal_locations = pd.read_csv(optimal_path)

# 3. 충전소 위치: grid_id로 격자 중심점 merge
stations_with_coords = pd.merge(
    charging_stations,
    grids[['grid_id', 'center_lat', 'center_lon']],
    on='grid_id',
    how='left'
)
stations_with_coords = stations_with_coords.dropna(subset=['center_lat', 'center_lon'])

# 4. 추천 입지: selected=1만 필터링
if 'selected' in optimal_locations.columns:
    optimal_selected = optimal_locations[optimal_locations['selected'] == 1].copy()
else:
    optimal_selected = optimal_locations.copy()

# 5. 지도 중심 좌표 설정 (추천 입지 평균값)
center_lat = optimal_selected['center_lat'].mean()
center_lon = optimal_selected['center_lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='CartoDB positron')

# 6. 기존 충전소 레이어 (파란색)
charging_layer = folium.FeatureGroup(name='기존 충전소', show=True)
for _, row in stations_with_coords.iterrows():
    name = row.get('충전소명', row.get('충전소ID', row.get('name', '')))
    folium.CircleMarker(
        location=[row['center_lat'], row['center_lon']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"기존 충전소: {name}"
    ).add_to(charging_layer)
charging_layer.add_to(m)

# 7. 추천 입지 레이어 (빨간색)
optimal_layer = folium.FeatureGroup(name='추천 입지(최적화)', show=True)
for _, row in optimal_selected.iterrows():
    grid_id = row.get('grid_id', '')
    score = row.get('predicted_demand_score', row.get('optimization_score', ''))
    folium.CircleMarker(
        location=[row['center_lat'], row['center_lon']],
        radius=7,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.85,
        popup=f"추천 입지: {grid_id}<br>최적화 점수: {score}"
    ).add_to(optimal_layer)
optimal_layer.add_to(m)

# 8. 서울 경계선 geojson 오버레이 (선택)
boundary_path = os.path.join(project_root, 'data', 'seoul_boundary.geojson')
if os.path.exists(boundary_path):
    folium.GeoJson(
        boundary_path,
        name='Seoul Boundary',
        style_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'fill': False
        }
    ).add_to(m)

# 9. 레이어 컨트롤
folium.LayerControl(position='topright', collapsed=False).add_to(m)

m


FileNotFoundError: 파일이 존재하지 않습니다: d:\projects\2025-1\빅데이터프로그래밍\data\processed\charging_stations_seoul_gridded.csv